In [1]:
from datasets import load_dataset
from adapters import AutoAdapterModel

from transformers import BertConfig
from transformers import BertTokenizer
import wandb

In [2]:
wandb.login(key="d1209ac0e4a568567e407d3afef99099ea87a4b1")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johntoro (ai4sw). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/huypn16/.netrc


True

In [3]:
%env WANDB_PROJECT=peft
%env WANDB_ENTITY=SC4001

env: WANDB_PROJECT=peft
env: WANDB_ENTITY=SC4001


In [4]:
dataset = load_dataset("yelp_review_full")
dataset.num_rows

Path, databilder:  yelp_review_full


{'train': 650000, 'test': 50000}

In [5]:

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

dataset = dataset.map(encode_batch, batched=True)
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from adapters import IA3Config, SeqBnConfig, LoRAConfig, ReftConfig, CompacterConfig

ia3_config = IA3Config()
seq_bn_config = SeqBnConfig()
lora_config = LoRAConfig()
compacter_config = CompacterConfig()
reft_config = ReftConfig(layers="all", prefix_positions=3, suffix_positions=0, r=1, orthogonality=True)


In [8]:
config = BertConfig.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
)
model = AutoAdapterModel.from_pretrained(
    "bert-base-uncased",
    config=config,
)

BertAdapterModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should

In [9]:
runname = "bert-rotten_tomatoes-compacter"
model.add_adapter("rotten_tomatoes", config=ia3_config)
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
)

# Activate the adapter
model.train_adapter("rotten_tomatoes")

# calculate the number of training parameters
total = 0
for p in model.parameters():
    if p.requires_grad:
        total += p.numel()
print(f"Total number of trainable parameters: {total}")

Total number of trainable parameters: 1270076


In [10]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    run_name=runname
)

def compute_accuracy(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [11]:
trainer.train()

[2024-11-18 04:51:28,633] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/datadrive5/huypn16/anaconda3/envs/ana/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/datadrive5/huypn16/anaconda3/envs/ana/compiler_compat/ld: /usr/local/cuda-12.5/lib64/libcufile.so: undefined reference to `dlvsym'
/datadrive5/huypn16/anaconda3/envs/ana/compiler_compat/ld: /usr/local/cuda-12.5/lib64/libcufile.so: undefined reference to `dlopen'
/datadrive5/huypn16/anaconda3/envs/ana/compiler_compat/ld: /usr/local/cuda-12.5/lib64/libcufile.so: undefined reference to `dlclose'
/datadrive5/huypn16/anaconda3/envs/ana/compiler_compat/ld: /usr/local/cuda-12.5/lib64/libcufile.so: undefined reference to `dlerror'
/datadrive5/huypn16/anaconda3/envs/ana/compiler_compat/ld: /usr/local/cuda-12.5/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


AttributeError: 'AdamW' object has no attribute 'train'

wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_general_ci impossible for parameter (<Response [422]>)
wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_general_ci impossible for parameter (<Response [422]>)
wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_general_ci impossible for parameter (<Response [422]>)
wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_general_ci impossible for parameter (<Response [422]>)
wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_general_ci impossible for parameter (<Response [422]>)
wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_gener

In [ ]:
trainer.evaluate(dataset["validation"])

/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.279348224401474,
 'eval_acc': 0.8846153846153846,
 'eval_runtime': 6.3115,
 'eval_samples_per_second': 168.897,
 'eval_steps_per_second': 2.693,
 'epoch': 6.0}